In [1]:
!git clone https://github.com/s-nlp/detox

Cloning into 'detox'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 186 (delta 54), reused 118 (delta 14), pack-reused 0
Receiving objects: 100% (186/186), 17.92 MiB | 18.55 MiB/s, done.
Resolving deltas: 100% (54/54), done.


## Setting up

In [2]:
! pip install -r detox/requirements.txt -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cmudict 1.0.13 requires importlib-metadata<6.0.0,>=5.1.0, but you have importlib-metadata 3.10.1 which is incompatible.
dask 2023.7.0 requires importlib-metadata>=4.13.0, but you have importlib-metadata 3.10.1 which is incompatible.
dask-cuda 23.6.0 requires dask==2023.3.2, but you have dask 2023.7.0 which is incompatible.
dask-cudf 23.6.1 requires dask==2023.3.2, but you have dask 2023.7.0 which is incompatible.
distributed 2023.3.2.1 requires dask==2023.3.2, but you have dask 2023.7.0 which is incompatible.
google-cloud-pubsublite 1.8.2 requires overrides<7.0.0,>=6.0.1, but you have overrides 3.1.0 which is incompatible.
jupyterlab-lsp 4.2.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
keyring 24.2.0 req

In [3]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [4]:
# adding the path to the condebert code root
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/kaggle/working/detox/emnlp2021/style_transfer/condBERT


In [5]:
from importlib import reload
import condbert
reload(condbert)
from condbert import CondBertRewriter
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
device = torch.device('cpu')

### Condbert Compile Vocab

In [7]:
VOCAB_DIRNAME = 'vocabularies' 

In [8]:
from condbert import CondBertRewriter
from choosers import EmbeddingSimilarityChooser
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

#### Load Bert

In [10]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
import os
from tqdm.auto import tqdm, trange

In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda:0')

In [13]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [14]:
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.to(device);

#### Preparing the vocabularires

In [16]:
if not os.path.exists(VOCAB_DIRNAME):
    os.makedirs(VOCAB_DIRNAME)

#### 2.1 Preparing the DRG-like vocabularies

In [17]:
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, tox_corpus, norm_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        tox_count_matrix = self.vectorizer.fit_transform(tox_corpus)
        self.tox_vocab = self.vectorizer.vocabulary_
        self.tox_counts = np.sum(tox_count_matrix, axis=0)

        norm_count_matrix = self.vectorizer.fit_transform(norm_corpus)
        self.norm_vocab = self.vectorizer.vocabulary_
        self.norm_counts = np.sum(norm_count_matrix, axis=0)

    def salience(self, feature, attribute='tox', lmbda=0.5):
        assert attribute in ['tox', 'norm']
        if feature not in self.tox_vocab:
            tox_count = 0.0
        else:
            tox_count = self.tox_counts[0, self.tox_vocab[feature]]

        if feature not in self.norm_vocab:
            norm_count = 0.0
        else:
            norm_count = self.norm_counts[0, self.norm_vocab[feature]]

        if attribute == 'tox':
            return (tox_count + lmbda) / (norm_count + lmbda)
        else:
            return (norm_count + lmbda) / (tox_count + lmbda)

In [1]:
import pandas as pd

df = pd.read_csv('../data/interim/data2.csv')

In [23]:
tox = df['toxic']
neu = df['neutral']

In [2]:
neg_out_name = "../detoxs_rus/emnlp2021/style_transfer/condBERT/vocab/negative-words.txt"
pos_out_name = "../detoxs_rus/emnlp2021/style_transfer/condBERT/vocab/positive-words.txt"

In [26]:
from collections import Counter
c = Counter()

with open(neg_out_name, 'r') as neg_out, open(pos_out_name, 'r') as pos_out:
    existant_pos_words = pos_out.readlines()
    for line in existant_pos_words:
        for tok in line.strip().split():
            c[tok] += 1
    existant_neg_words = neg_out.readlines()
    for line in existant_neg_words:
        for tok in line.strip().split():
            c[tok] += 1

for fn in [tox, neu]:
        for line in fn:
            for tok in line.strip().split():
                c[tok] += 1
print(len(c))

281442


In [27]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

281442


In [31]:
# with open(tox_corpus_path, 'r') as tox_corpus, open(norm_corpus_path, 'r') as norm_corpus:
corpus_tox = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in tox]
corpus_norm = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in neu]

In [32]:
neg_out_name = VOCAB_DIRNAME + '/negative-words.txt'
pos_out_name = VOCAB_DIRNAME + '/positive-words.txt'

In [33]:
threshold = 4

In [34]:
sc = NgramSalienceCalculator(corpus_tox, corpus_norm, False)
seen_grams = set()

with open(neg_out_name, 'w') as neg_out, open(pos_out_name, 'w') as pos_out:
    for gram in set(sc.tox_vocab.keys()).union(set(sc.norm_vocab.keys())):
        if gram not in seen_grams:
            seen_grams.add(gram)
            toxic_salience = sc.salience(gram, attribute='tox')
            polite_salience = sc.salience(gram, attribute='norm')
            if toxic_salience > threshold:
                neg_out.writelines(f'{gram}\n')
            elif polite_salience > threshold:
                pos_out.writelines(f'{gram}\n')

#### 2.2 Evaluating word toxicities with a logistic regression

In [37]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=10000))

In [38]:
X_train = corpus_tox + corpus_norm
y_train = [1] * len(corpus_tox) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train);

In [39]:
coefs = pipe[1].coef_[0]
coefs.shape

(101070,)

In [40]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [41]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

#### 2.3 Labelling BERT tokens by toxicity

In [42]:
from collections import defaultdict
toxic_counter = defaultdict(lambda: 1)
nontoxic_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_tox):
    for token in tokenizer.encode(text):
        toxic_counter[token] += 1
for text in tqdm(corpus_norm):
    for token in tokenizer.encode(text):
        nontoxic_counter[token] += 1

100%|██████████| 577777/577777 [04:33<00:00, 2108.71it/s]


In [43]:
token_toxicities = [toxic_counter[i] / (nontoxic_counter[i] + toxic_counter[i]) for i in range(len(tokenizer.vocab))]

In [44]:
with open(VOCAB_DIRNAME + '/token_toxicities.txt', 'w') as f:
    for t in token_toxicities:
        f.write(str(t))
        f.write('\n')

#### 3. Setting up the model

In [45]:
with open(VOCAB_DIRNAME + "/negative-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))

with open(VOCAB_DIRNAME + "/positive-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

In [46]:
import pickle
with open(VOCAB_DIRNAME + '/word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [47]:
token_toxicities = []
with open(VOCAB_DIRNAME + '/token_toxicities.txt', 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

In [48]:
def adjust_logits(logits, label=0):
    return logits - token_toxicities * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=predictor,
)

In [49]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

2023-11-05 18:22:09,282 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpnn88uuz2


100%|██████████| 160000128/160000128 [00:08<00:00, 19695200.97B/s]

2023-11-05 18:22:18,080 copying /tmp/tmpnn88uuz2 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy
2023-11-05 18:22:18,201 removing temp file /tmp/tmpnn88uuz2


2023-11-05 18:22:18,638 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmp8j940ugj


100%|██████████| 21494764/21494764 [00:01<00:00, 12222875.42B/s]

2023-11-05 18:22:20,813 copying /tmp/tmp8j940ugj to cache at /root/.flair/embeddings/glove.gensim


2023-11-05 18:22:20,832 removing temp file /tmp/tmp8j940ugj


#### 4. Finally, the inference

In [50]:
print(editor.translate('You are idiot!', prnt=False))

you are mistake !


In [51]:
print(editor.replacement_loop('You are stupid!', verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))

you are so beautiful !


Parameters that could be tuned:

* The coeffincient in adjust_logits - the larger it is, the more the model avoids toxic words
* The coefficient in EmbeddingSimilarityChooser - the larger it is, the more the model tries to preserve content
* n_tokens - how many words can be generated from one
* n_top - how many BERT hypotheses are reranked

In [54]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [55]:
test_output = []

lines = list(test_data['toxic'])
for i, tx in enumerate(tqdm(lines)):
    inp = tx.strip()
    out = editor.translate(inp, prnt=False)
    test_output += out

100%|██████████| 115556/115556 [36:28<00:00, 52.80it/s]


In [ ]:
with open('results.txt', 'w') as file:
    for item in test_output:
        file.write("%s\n" % item)